In [1]:
import numpy as np
import pandas as pd

#Preprocessing libraries
import preprocessor as p
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer
from ekphrasis.classes.segmenter import Segmenter
from datetime import datetime, timedelta 
#For the regular expression
import itertools
import re
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ayush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# # save filepath to variable for easier access
# tweets_file_path = os.path.expanduser('~/Downloads/covid19_tweets.csv')
# # read the data and store data 
# tweets_data = pd.read_csv(tweets_file_path)

# data_subset = tweets_data.head(500)
# # tweets_data['text']
# tweets_data.describe()    
# data_subset.to_csv (r'~/Downloads/export_dataframe.csv', index = False, header=True)

In [3]:
# save filepath to variable for easier access
tweets_file_path = 'C:/Users/ayush/Documents/Trento/Data Mining/Project/data/covid19_tweets_sort.csv'
# read the data and store data 
tweets_data = pd.read_csv(tweets_file_path)

In [3]:
set_of_dates = sorted(set(tweets_data.date))
print(set_of_dates[0])
print(set_of_dates[-1])

date= pd.to_datetime(set_of_dates)
min_time_delta= timedelta(days = 0, hours=0, minutes=0, seconds=0)
# print(set_of_dates[1]-set_of_dates[0])
# print((set_of_dates[1]-set_of_dates[0])>min_time_delta)
print((date[1]-date[0]))
print((date[1]-date[0])>min_time_delta)

2020-07-25 12:11:07
2020-07-25 12:27:21
0 days 00:00:01
True


In [4]:
## Drop duplicates
tweets_data.drop_duplicates(inplace=True, subset="text")


In [4]:
for i,v in enumerate(tweets_data['text']):
    tweets_data.loc[i,'text_clean'] = p.clean(v)

In [5]:
def preprocess_data(data):
 #Removes Numbers
    data = data.astype(str).str.replace('\d+', '')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer =  TweetTokenizer()
 
    def lemmatize_text(text):
        return [(lemmatizer.lemmatize(w)) for w \
                       in w_tokenizer.tokenize((text))]
    def remove_punctuation(words):
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', (word))
            if new_word != '' and len(new_word) > 2:
                new_words.append(new_word)
        return new_words
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    return pd.DataFrame(words)

In [6]:
pre_tweets = preprocess_data(tweets_data['text_clean'])
print(pre_tweets)
tweets_data['text_clean'] = pre_tweets
stop_words = set(nltk.corpus.stopwords.words('english'))
tweets_data['text_clean'] = tweets_data['text_clean'].apply(lambda x: [item for item in \
                                    x if item not in stop_words])

                                               text_clean
0       [some, more, excellent, work, from, our, assis...
1       [donald, trump, hold, telerally, campaign, fir...
2       [all, play, part, keeping, our, community, saf...
3       [hello, kindly, help, the, child, your, consti...
4       [contemplate, sometimes, all, you, have, for, ...
...                                                   ...
179103  [hearing, lot, about, what, great, piece, this...
179104  [day, having, group, session, under, the, sess...
179105  [just, wondering, the, result, test, will, del...
179106  [while, the, whole, country, divided, between,...
179107  [have, updated, our, guide, now, include, coro...

[179108 rows x 1 columns]


In [7]:
seg_tw = Segmenter(corpus="twitter")
a = []
for i in range(len(tweets_data)):
#     if tweets_data['text_clean'][i] != a and pd.notna(tweets_data['text_clean'][i]):
    for elem in tweets_data['text_clean'][i]:
        listToStr1 = ' '.join([(elem) for elem in tweets_data['text_clean'][i]])
#     print(listToStr1)
    tweets_data.loc[i,'Segmented#'] = seg_tw.segment(listToStr1)

Reading twitter - 1grams ...
Reading twitter - 2grams ...


C:\Users\ayush\anaconda3\lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [8]:
print(tweets_data['text_clean'][9])
# tweets_data.head(15)

['stand', 'may', 'empty', 'biggest', 'fan', 'cheering', 'home', 'opening', 'day', 'grandfat']


In [9]:
tweets_data['segmented']=tweets_data['Segmented#']
tweets_data['segmented'][0:10]
# tweets_data.head()

0    excellent  work  assistant  director  jennifer...
1    donald   trump  hold telerally  campaign  firs...
2    play  part keeping  community  safe help slow ...
3    hello  kindly help child  constituency  contin...
4    contemplate  sometimes swim  meditating  nappi...
5    year  parent excited  getting  internet  conne...
6    week  commission  passed  revised local  healt...
7    opponent  doesnt care thats  that s  tweet vot...
8    chief  public health  officer  warns pandemic ...
9    stand  may empty  biggest  fan  cheering  home...
Name: segmented, dtype: object

In [10]:

# Convert hashtags to lowercase
tweets_data['hashtags'] = tweets_data.hashtags.str.lower()

tweets_data['hashtags'] = tweets_data.hashtags.fillna('')

print(tweets_data['hashtags'])

0                                                          
1                                                          
2                                                          
3                                                          
4                                                          
                                ...                        
179103                                          ['covid19']
179104       ['shapersummit20', 'skills', 'designthinking']
179105                                          ['covid19']
179106    ['ssrcase', 'mann_ki_nahi_students_ki_baat', '...
179107                                       ['athomecare']
Name: hashtags, Length: 179108, dtype: object


In [11]:
# tweets_data['hash']=tweets_data['hashtags#']
tweets_data['hashtags'].dtypes


dtype('O')

In [12]:
tweets_data.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,text_clean,Segmented#,segmented
0,Center on Trauma and Adversity,"Cleveland, OH",Reducing the impact of #trauma and #adversity ...,26-10-2017 18:48,6292,446,3619,False,24-07-2020 23:47,Some more excellent work from our Assistant Di...,,Twitter for iPhone,False,"[excellent, work, assistant, director, jennife...",excellent work assistant director jennifer...,excellent work assistant director jennifer...
1,IAM Platform,Worldwide,Curation | Tools | Tips | Services\n\nIAM Plat...,13-12-2010 21:24,17816,14941,65440,False,24-07-2020 23:47,Donald Trump holds ‘tele-rally’ in campaign fi...,,IAMBLOG2TWITTER,False,"[donald, trump, hold, telerally, campaign, fir...",donald trump hold telerally campaign firs...,donald trump hold telerally campaign firs...
2,"CA Business, Consumer Services and Housing Agency","Sacramento, CA",We are responsible for licensing and oversight...,15-10-2012 18:10,1351,278,278,False,24-07-2020 23:47,We all play a part in keeping our communities ...,,Twitter Web App,False,"[play, part, keeping, community, safe, help, s...",play part keeping community safe help slow ...,play part keeping community safe help slow ...
3,enjuba,East Africa,We are a children’s education organisation aim...,27-02-2013 13:59,10260,6291,4831,False,24-07-2020 23:47,Hello @onyukandrew1 Kindly help the children i...,,Twitter for iPhone,False,"[hello, kindly, help, child, constituency, con...",hello kindly help child constituency contin...,hello kindly help child constituency contin...
4,Jordi Ortega,"Los Angeles, CA","Story miner: Innovation, Ventures @StateFarm. ...",15-11-2013 21:57,1475,4417,14809,False,24-07-2020 23:47,"“CONTEMPLATE.” Sometimes, all you have to do i...",,Twitter for iPhone,False,"[contemplate, sometimes, swim, meditating, nap...",contemplate sometimes swim meditating nappi...,contemplate sometimes swim meditating nappi...


In [13]:
clean_tweets_data = tweets_data[['date', 'text_clean', 'hashtags','segmented']].copy()
clean_tweets_data = pd.DataFrame([tweets_data.date, tweets_data.text_clean, tweets_data.hashtags, tweets_data.segmented]).transpose()
clean_tweets_data.head(5)

,date,text_clean,hashtags,segmented
0,24-07-2020 23:47,"[excellent, work, assistant, director, jennife...",,excellent work assistant director jennifer...
1,24-07-2020 23:47,"[donald, trump, hold, telerally, campaign, fir...",,donald trump hold telerally campaign firs...
2,24-07-2020 23:47,"[play, part, keeping, community, safe, help, s...",,play part keeping community safe help slow ...
3,24-07-2020 23:47,"[hello, kindly, help, child, constituency, con...",,hello kindly help child constituency contin...
4,24-07-2020 23:47,"[contemplate, sometimes, swim, meditating, nap...",,contemplate sometimes swim meditating nappi...


In [14]:
# clean_tweets_data['merged_text'] = clean_tweets_data['text_clean'].astype(str) + clean_tweets_data['hashtags'].fillna('').astype(str)

In [15]:
clean_tweets_data.head(5)

,date,text_clean,hashtags,segmented
0,24-07-2020 23:47,"[excellent, work, assistant, director, jennife...",,excellent work assistant director jennifer...
1,24-07-2020 23:47,"[donald, trump, hold, telerally, campaign, fir...",,donald trump hold telerally campaign firs...
2,24-07-2020 23:47,"[play, part, keeping, community, safe, help, s...",,play part keeping community safe help slow ...
3,24-07-2020 23:47,"[hello, kindly, help, child, constituency, con...",,hello kindly help child constituency contin...
4,24-07-2020 23:47,"[contemplate, sometimes, swim, meditating, nap...",,contemplate sometimes swim meditating nappi...


In [16]:
def str_to_array(text):
    str1=text.replace(']','').replace('[','')
    l=str1.replace("'",'').split(",")
    l1 = [item.strip(' ') for item in l]
    return l1

In [17]:
for i,v in enumerate(clean_tweets_data['text_clean'].astype(str)):
        clean_tweets_data['text_clean'][i] = (str_to_array(v))

In [18]:
for i,v in enumerate(clean_tweets_data['hashtags'].astype(str)):
        clean_tweets_data['hashtags'][i] = (str_to_array(v))

In [19]:
clean_tweets_data['merged_text'] = clean_tweets_data['text_clean'] + clean_tweets_data['hashtags']

In [20]:
print(clean_tweets_data['merged_text'])
clean_tweets_data['segmented'].dtypes

0         [excellent, work, assistant, director, jennife...
1         [donald, trump, hold, telerally, campaign, fir...
2         [play, part, keeping, community, safe, help, s...
3         [hello, kindly, help, child, constituency, con...
4         [contemplate, sometimes, swim, meditating, nap...
                                ...                        
179103    [hearing, lot, great, piece, timpsons, handled...
179104    [day, group, session, session, using, create, ...
179105    [wondering, result, test, delayed, passed, alr...
179106    [whole, country, divided, increasing, ssrcase,...
179107    [updated, guide, include, coronvirus, section,...
Name: merged_text, Length: 179108, dtype: object


dtype('O')

In [21]:
print(clean_tweets_data['merged_text'][0])
clean_tweets_data['merged_text'].dtypes

['excellent', 'work', 'assistant', 'director', 'jennifer', 'king', 'informed', 'decision', '']


dtype('O')

In [22]:
# f = lambda x: 'word_{}'.format(x + 1)
# new_df = pd.concat([pd.DataFrame(
#     clean_tweets_data.merged_text.values.tolist(),
#     clean_tweets_data.index, dtype=object).fillna('').rename(columns=f),
#     clean_tweets_data.date],axis=1)

In [23]:
f = lambda x: 'word_{}'.format(x + 1)
new_df = (pd.DataFrame
    (clean_tweets_data.merged_text.values.tolist(),
     clean_tweets_data.index, dtype=object).fillna('').rename(columns=f))

In [24]:
new_df

,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,word_10,word_11,word_12,word_13,word_14,word_15,word_16,word_17,word_18,word_19,word_20
0,excellent,work,assistant,director,jennifer,king,informed,decision,,,,,,,,,,,,
1,donald,trump,hold,telerally,campaign,first,amid,coronavirus,pandemic,read,,,,,,,,,,
2,play,part,keeping,community,safe,help,slow,spread,protect,others,weekend,wearing,,,,,,,,
3,hello,kindly,help,child,constituency,continue,learning,getting,boo,,,,,,,,,,,
4,contemplate,sometimes,swim,meditating,napping,pagoda,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179103,hearing,lot,great,piece,timpsons,handled,unfortu,covid19,,,,,,,,,,,,
179104,day,group,session,session,using,create,shapersummit20,skills,designthinking,,,,,,,,,,,
179105,wondering,result,test,delayed,passed,alreadyid,like,covid19,,,,,,,,,,,,
179106,whole,country,divided,increasing,ssrcase,mann_ki_nahi_students_ki_baat,covid19,,,,,,,,,,,,,


In [25]:
final_df = pd.concat([clean_tweets_data.date, new_df],axis=1)

In [26]:
final_df

,date,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,...,word_11,word_12,word_13,word_14,word_15,word_16,word_17,word_18,word_19,word_20
0,24-07-2020 23:47,excellent,work,assistant,director,jennifer,king,informed,decision,,...,,,,,,,,,,
1,24-07-2020 23:47,donald,trump,hold,telerally,campaign,first,amid,coronavirus,pandemic,...,,,,,,,,,,
2,24-07-2020 23:47,play,part,keeping,community,safe,help,slow,spread,protect,...,weekend,wearing,,,,,,,,
3,24-07-2020 23:47,hello,kindly,help,child,constituency,continue,learning,getting,boo,...,,,,,,,,,,
4,24-07-2020 23:47,contemplate,sometimes,swim,meditating,napping,pagoda,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179103,30-08-2020 09:07,hearing,lot,great,piece,timpsons,handled,unfortu,covid19,,...,,,,,,,,,,
179104,30-08-2020 09:07,day,group,session,session,using,create,shapersummit20,skills,designthinking,...,,,,,,,,,,
179105,30-08-2020 09:07,wondering,result,test,delayed,passed,alreadyid,like,covid19,,...,,,,,,,,,,
179106,30-08-2020 09:07,whole,country,divided,increasing,ssrcase,mann_ki_nahi_students_ki_baat,covid19,,,...,,,,,,,,,,


In [27]:
final_df.to_csv (r'C:/Users/ayush/Documents/Trento/Data Mining/Project/data/final_df_space_remove.csv', index = False, header=True)